### MNIST For ML Beginnersの内容をtensorboardで表示させる

https://www.tensorflow.org/get_started/mnist/beginners

#### TensorBoardを使用する際の考慮点
1. With構文を用いたグラフコンテキスト内に、Placeholder、Variable、計算値の定義を記載する。

2. With構文による「name_scope」を用いて、入力層、隠れ層、出力層などに構成要素をグループ化する。

3. ネットワークグラフに付与するラベル名をコード内で指定する。

4. グラフに表示するパラメータを宣言して、「summary.FileWriter」オブジェクトでデータを書き出す。

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 手書き文字データを取得。
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
"""
With構文を用いたグラフコンテキスト内に、Placeholder、Variable、計算値の定義を記載するため、
クラスを生成する。
""" 
class MnistBeginnerModel:
    # コンストラクタ
    def __init__ (self):
        # グラグコンテキストを開始する。
        with tf.Graph().as_default():
            # モデルを準備する関数を呼び出す。
            self.prepare_model()
            # セッションを準備する関数を呼び出す。
            self.prepare_session()
    
    # モデルを準備する
    def prepare_model(self):
        
        # 以下で入力や出力などの構成単位でネームスコープを定義する。
        
        # inputとなる項目をグループ化
        with tf.name_scope("input"):
            # placeholderは、値（テンソル）が入力される場所
            # [None, 784]は2次元のテンソル。Noneは次元の指定なしの意味
            x = tf.placeholder(tf.float32 , [None, 784] , name = "x")
            
            # 重みを定義
            # Variableは変更可能なテンソル
            w = tf.Variable(tf.zeros([784, 10]) , name = "weight")
            
            # バイアスを定義
            b = tf.Variable(tf.zeros([10]) , name = "baias")
        
        with tf.name_scope("output"):
            # x * W + bを計算。
            # ソフトマックス関数を用いることで、xW+bの各要素を0~1に変換している
            y = tf.nn.softmax(tf.matmul(x, w) + b , name = "softmax")
        
        with tf.name_scope("optimizer"):
            # 正解ラベル
            y_ = tf.placeholder(tf.float32, [None, 10] , name = "label")
            
            #  クロスエントロピー(コスト関数)
            # (教師データの入力からNNを用いて求めた出力)とy_（教師データの正解）のクロスエントロピーを計算して、正解度をチェック
            cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) , reduction_indices=[1]) , name = "loss")
            
            # 最急降下法を用いてクロスエントロピーの最小化を図る
            train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
            
        with tf.name_scope("evaluating"):
            # 学習させた後、未知のデータを正しく分類できるかの評価基準を設定
            correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
            
            # correct_predictionは、[True, False, True, True,　・・・]といったデータ構造なので、これを0(False),1(True)に対応付けて、その平均値を計算
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32) , name = "accuracy")
            
        
        # tf.summary.scalar()は誤差関数や正解率のような、単一の値を取る要素について変化を折れ線グラフとして表示する。
        
        # クロスエントロピーを折れ線グラフとして表示させるため、定義
        tf.summary.scalar("loss" , cross_entropy)
        
        # 正解の平均値を折れ線グラフとして表示する。
        tf.summary.scalar("accuracy" , accuracy)
        
        # tf.summary.histogram'()は複数の要素を含む多次元リストについて、値の分布を表示する。
        
        # 重みをヒストグラムで表示する。
        tf.summary.histogram("weight" , w)
        
        # クラスの外部から参照する必要がある変数をインスタンス変数として公開する。
        self.x ,self.y , self.y_ = x , y , y_
        self.train_step = train_step
        self.loss = cross_entropy
        self.accuracy = accuracy
        
    def prepare_session(self):
        #   Sessionを初期化して開始
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        # 定義した表示項目をマージ
        summary = tf.summary.merge_all()
        # 出力先を指定して、書き出し用のオブジェクトを定義
        writer = tf.summary.FileWriter("temp/minist_tensorboard_data" , sess.graph)
        
        # クラスの外部から参照する必要がある変数をインスタンス変数として公開する。
        self.sess = sess
        self.summary = summary
        self.writer = writer

In [4]:
# 以前に実行したデータが残っているとTensorBoardの出力が乱れることがあるので、フォルダを削除しておく。
!rm -rf temp/minist_tensorboard_data

In [5]:
mnist_model = MnistBeginnerModel()

i = 0
for _ in  range(2000):
    i += 1
    batch_xs, batch_ys = mnist.train.next_batch(100)
    mnist_model.sess.run(mnist_model.train_step, feed_dict={mnist_model.x: batch_xs, mnist_model.y_: batch_ys})
    
    if i % 100 ==0:
        summary, loss_val , acc_val = mnist_model.sess.run([mnist_model.summary , mnist_model.loss , mnist_model.accuracy],
                                                          feed_dict={mnist_model.x: mnist.test.images, mnist_model.y_: mnist.test.labels})
        print('Step : %d , Loss :  %f , Accuracy : %f' %(i,loss_val, acc_val))
        mnist_model.writer.add_summary(summary , i)

Step : 100 , Loss :  0.385322 , Accuracy : 0.894300
Step : 200 , Loss :  0.349057 , Accuracy : 0.903700
Step : 300 , Loss :  0.326020 , Accuracy : 0.908300
Step : 400 , Loss :  0.333351 , Accuracy : 0.903700
Step : 500 , Loss :  0.320673 , Accuracy : 0.909400
Step : 600 , Loss :  0.312486 , Accuracy : 0.909500
Step : 700 , Loss :  0.295034 , Accuracy : 0.916100
Step : 800 , Loss :  0.300036 , Accuracy : 0.915600
Step : 900 , Loss :  0.296166 , Accuracy : 0.917500
Step : 1000 , Loss :  0.292919 , Accuracy : 0.919300
Step : 1100 , Loss :  0.291226 , Accuracy : 0.918700
Step : 1200 , Loss :  0.288829 , Accuracy : 0.917100
Step : 1300 , Loss :  0.289277 , Accuracy : 0.917600
Step : 1400 , Loss :  0.289422 , Accuracy : 0.916300
Step : 1500 , Loss :  0.291647 , Accuracy : 0.918700
Step : 1600 , Loss :  0.285784 , Accuracy : 0.921600
Step : 1700 , Loss :  0.284413 , Accuracy : 0.916800
Step : 1800 , Loss :  0.285138 , Accuracy : 0.920100
Step : 1900 , Loss :  0.279823 , Accuracy : 0.920900
St

In [6]:
# ターミナルで以下を打鍵
# tensorboard --logdir=temp/minist_tensorboard_data

# もし表示した先が何も出力されていなければ、pwdで現在地を調べる。
# logdirの先にファイルがなくてもTensorBoardは起動する。